In [1]:
import os
import numpy as np
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from skimage.io import imread
from skimage.transform import resize

In [2]:
train_path = r"D:\Skin_Cancer\cancer\melanoma_cancer_dataset\train"
test_path = r"D:\Skin_Cancer\cancer\melanoma_cancer_dataset\test"

In [ ]:

categories = ['benign','malignant']
def load_data(path,categories):
    x_data,y_data = [],[]
    for category in categories:
        label = categories.index(category)
        img_paths = glob.glob(os.path.join(path,category,"*.jpg")) 
        for img_path in img_paths:
            img_array = imread(img_path)
            img_resized = resize(img_array, (150, 150, 3))
            x_data.append(img_resized)
            y_data.append(label)
        print(f"Loaded {len(img_paths)} images from category: {category}")
    return np.array(x_data) / 255.0, np.array(y_data)

In [ ]:
x_train,y_train = load_data(train_path,categories)
x_test,y_test = load_data(test_path,categories)

y_train = to_categorical(y_train,num_classes=2)
y_test = to_categorical(y_test,num_classes=2)

Loaded 5000 images from category: benign
Loaded 4605 images from category: malignant
Loaded 500 images from category: benign
Loaded 500 images from category: malignant


In [ ]:

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator()
val_generator = val_datagen.flow(x_test,y_test,batch_size=32)

model = Sequential([
    Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)),
    MaxPool2D(2,2),
    Conv2D(64,(3,3),activation='relu'),
    MaxPool2D(2, 2),
    Conv2D(128, (3, 3),activation='relu'),
    MaxPool2D(2,2),
    Flatten(),
    Dense(128,activation='relu'),
    Dropout(0.5),
    Dense(2,activation='softmax')
])


model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

callbacks = [
    ReduceLROnPlateau(monitor='val_loss',factor=0.5,patience=3,verbose=1,min_lr=1e-6),
    ModelCheckpoint("model.keras",monitor='val_accuracy',save_best_only=True,verbose=1)
]


history = model.fit(
    datagen.flow(x_train,y_train,batch_size=32),
    validation_data=val_generator,
    epochs=50,
    callbacks=callbacks
)

d:\Skin_Cancer\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
d:\Skin_Cancer\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
301/301 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step - accuracy: 0.5846 - loss: 0.6588
Epoch 1: val_accuracy improved from -inf to 0.76300, saving model to model.keras
301/301 ━━━━━━━━━━━━━━━━━━━━ 108s 350ms/step - accuracy: 0.5848 - loss: 0.6586 - val_accuracy: 0.7630 - val_loss: 0.4804 - learning_rate: 1.0000e-04
Epoch 2/50
301/301 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step - accuracy: 0.7418 - loss: 0.5241
Epoch 2: val_accuracy improved from 0.76300 to 0.80200, saving model to model.keras
301/301 ━━━━━━━━━━━━━━━━━━━━ 97s 321ms/step - accuracy: 0.7419 - loss: 0.5241 - val_accuracy: 0.8020 - val_loss: 0.4126 - learning_rate: 1.0000e-04
Epoch 3/50
301/301 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step - accuracy: 0.7672 - loss: 0.4593
Epoch 3: val_accuracy improved from 0.80200 to 0.81600, saving model to model.keras
301/301 ━━━━━━━━━━━━━━━━━━━━ 89s 296ms/step - accuracy: 0.7673 - loss: 0.4593 - val_accuracy: 0.8160 - val_loss: 0.3996 - learning_rate: 1.0000e-04
Epoch 4/50
301/301 ━━━━━━━━━━━━━━━━━━━━ 0s

In [6]:
model.evaluate(x_test,y_test)

32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - accuracy: 0.9194 - loss: 0.2283


[0.24523504078388214, 0.902999997138977]

In [1]:
y_pred=model.predict(x_test)
y_pred1=(y_pred.argmax(axis=1))

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred,axis=1) 
y_true_classes = np.argmax(y_test,axis=1)  

print(classification_report(y_true_classes,y_pred_classes))
